In [9]:
import duckdb
import pandas as pd
from pathlib import Path
from settings import settings
from alpha_vantage_schema import TABLE_SCHEMAS

In [10]:
conn = duckdb.connect(Path(settings.get("db_path", "data/alpha_vantage.db")))

In [11]:
# result = conn.execute("SELECT * FROM INSIDER_TRANSACTIONS WHERE symbol = 'GOOG'").df()
result = conn.execute("TRUNCATE TABLE INSIDER_TRANSACTIONS").df()
print(result)
# df = pd.read_parquet("data/files/INSIDER_TRANSACTIONS/symbol_GOOG.parquet", engine='pyarrow')
# print(df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     Count
0  1194882


In [12]:
conn.close()